# Torrents To Google Drive Downloader
*Python app to save content from torrent directly to google drive.*

### Step 1: Mount Google Drive
To stream files we need to get access to write on that drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Step 2: Install libtorrent library
Libtorrent is a feature complete C++ bittorrent implementation focusing on efficiency and scalability.
https://www.libtorrent.org/

In [ ]:
!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent

### Step 3: Paste the Magnet Link in the input box
Variable **link** stores the link string.

In [ ]:
link = input("PASTE TORRENT/MAGNET LINK HERE \n") # PASTE TORRENT/MAGNET LINK HERE

### Step 4: Download torrent

In [ ]:
import libtorrent as lt
import time
import datetime

ses = lt.session()
ses.listen_on(6881, 6891) #here we are defining the ports on which BitTorrent works
params = {
    'save_path': '/content/drive/My Drive/Torrent/',
    'storage_mode': lt.storage_mode_t(2)}
# params means parameters
# In paras we are defining the location on drive where we want to save the downloaded files
# we also define storage mode, in this case we are using "sparse allocation mode"
# The sparse allocation, sparse files are used, and pieces are downloaded directly to where they belong. This is the recommended (and default) mode.
print(link)

handle = lt.add_magnet_uri(ses, link, params)
ses.start_dht()

begin = time.time()
print(datetime.datetime.now())

print ('Downloading Metadata...')
while (not handle.has_metadata()):
    time.sleep(1)
print ('Got Metadata, Starting Torrent Download...')

print("Starting", handle.name())

while (handle.status().state != lt.torrent_status.seeding):
    s = handle.status()
    state_str = ['queued', 'checking', 'downloading metadata', \
            'downloading', 'finished', 'seeding', 'allocating']
    print ('%.2f%% complete (down: %.1f kb/s up: %.1f kB/s peers: %d) %s ' % \
            (s.progress * 100, s.download_rate / 1000, s.upload_rate / 1000, \
            s.num_peers, state_str[s.state]))
    time.sleep(5)

end = time.time()
print(handle.name(), "COMPLETE")

print("Elapsed Time: ",int((end-begin)//60),"min :", int((end-begin)%60), "sec")

print(datetime.datetime.now())